<p style="font-weight:bold;"> <span style="font-size: 36px"> Test Aoc Configuration </span> </p>

In [0]:
#!import "../CalculationEngine"

# Workspace Initialization 

In [0]:
string novelties = 
@"@@Novelty
SystemName,DisplayName,Parent,Order
I,In Force,,1
N,New Business,,10
C,Combined,,20";

In [0]:
await Import.FromString(novelties).WithType<Novelty>().WithTarget(DataSource).ExecuteAsync()

In [0]:
var workspace = Workspace.CreateNew();
workspace.InitializeFrom(DataSource);

# Canonical Aoc Types and Configurations

In [0]:
string canonicalAocTypes = 
@"@@AocType,,,,,,,,,,,
SystemName,DisplayName,Parent,Order,,,,,,,,
BOP,Opening Balance,,10,,,,,,,,
MC,Model Correction,,20,,,,,,,,
PC,Portfolio Changes,,30,,,,,,,,
RCU,Reinsurance Coverage Update,PC,40,,,,,,,,
CF,Cash flow,,50,,,,,,,,
IA,Interest Accretion,,60,,,,,,,,
AU,Assumption Update,,70,,,,,,,,
FAU,Financial Assumption Update,,80,,,,,,,,
YCU,Yield Curve Update,FAU,90,,,,,,,,
CRU,Credit Risk Update,FAU,100,,,,,,,,
EV,Experience Variance,,110,,,,,,,,
WO,Write-Off,,120,,,,,,,,
CL,Combined Liabilities,,130,,,,,,,,
EA,Experience Adjustment,,140,,,,,,,,
AM,Amortization,,150,,,,,,,,
FX,FX Impact,,160,,,,,,,,
EOP,Closing Balance,,170,,,,,,,,";

In [0]:
string canonicalAocConfig = 
@"@@AocConfiguration,,,,,,,,,,,
AocType,Novelty,DataType,InputSource,FxPeriod,YcPeriod,CdrPeriod,ValuationPeriod,RcPeriod,Order,Year,Month
BOP,I,Optional,7,BeginningOfPeriod,BeginningOfPeriod,BeginningOfPeriod,BeginningOfPeriod,BeginningOfPeriod,10,1900,1
MC,I,Optional,4,BeginningOfPeriod,BeginningOfPeriod,BeginningOfPeriod,BeginningOfPeriod,BeginningOfPeriod,20,1900,1
RCU,I,Calculated,4,BeginningOfPeriod,BeginningOfPeriod,BeginningOfPeriod,BeginningOfPeriod,EndOfPeriod,30,1900,1
CF,I,Calculated,4,Average,NotApplicable,BeginningOfPeriod,Delta,EndOfPeriod,40,1900,1
IA,I,Calculated,5,Average,BeginningOfPeriod,BeginningOfPeriod,Delta,EndOfPeriod,50,1900,1
AU,I,Optional,4,EndOfPeriod,BeginningOfPeriod,BeginningOfPeriod,EndOfPeriod,EndOfPeriod,60,1900,1
YCU,I,CalculatedTelescopic,4,EndOfPeriod,EndOfPeriod,BeginningOfPeriod,EndOfPeriod,EndOfPeriod,70,1900,1
CRU,I,CalculatedTelescopic,4,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,80,1900,1
EV,I,Optional,4,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,90,1900,1
BOP,N,Optional,4,Average,EndOfPeriod,EndOfPeriod,BeginningOfPeriod,EndOfPeriod,95,1900,1
CF,N,Calculated,4,Average,NotApplicable,EndOfPeriod,Delta,EndOfPeriod,110,1900,1
IA,N,Calculated,4,Average,EndOfPeriod,EndOfPeriod,Delta,EndOfPeriod,120,1900,1
AU,N,Optional,4,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,130,1900,1
EV,N,Optional,4,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,140,1900,1
CF,C,Optional,2,Average,NotApplicable,NotApplicable,NotApplicable,NotApplicable,160,1900,1
WO,C,Optional,2,Average,NotApplicable,NotApplicable,NotApplicable,NotApplicable,170,1900,1
CL,C,Mandatory,4,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,180,1900,1
EA,C,Calculated,4,EndOfPeriod,NotApplicable,NotApplicable,NotApplicable,EndOfPeriod,190,1900,1
AM,C,Calculated,6,EndOfPeriod,NotApplicable,NotApplicable,NotApplicable,EndOfPeriod,200,1900,1
FX,C,Calculated,0,NotApplicable,NotApplicable,NotApplicable,NotApplicable,NotApplicable,210,1900,1
EOP,C,Calculated,6,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,EndOfPeriod,220,1900,1";

# Test

In [0]:
public async Task<ActivityLog> CheckAocConfigurationDefault(AocType[] newAocTypes)
{
    await workspace.DeleteAsync<AocType>( await workspace.Query<AocType>().ToArrayAsync() );
    await workspace.DeleteAsync<AocConfiguration>( await workspace.Query<AocConfiguration>().ToArrayAsync() );
    await workspace.CommitAsync();

    var aocTypeLog = await Import.FromString(canonicalAocTypes).WithType<AocType>().WithTarget(workspace).ExecuteAsync();
    aocTypeLog.Errors.Any().Should().Be(false);

    await workspace.UpdateAsync(newAocTypes);
    await workspace.CommitAsync();

    return await Import.FromString(canonicalAocConfig).WithFormat("AocConfiguration").WithTarget(workspace).ExecuteAsync();
}

## NewAoc < RCU

In [0]:
var aocConfigLog = await CheckAocConfigurationDefault( new[]{ new AocType{SystemName = "A1", DisplayName = "a1", Order = 25} } );
aocConfigLog.Errors.Any().Should().Be(false);

In [0]:
var newConfigExpected = (await workspace.Query<AocConfiguration>().ToArrayAsync()).First(x => 
                         x.AocType == AocTypes.MC && x.Novelty == Novelties.I) with {AocType = "A1", Order = 21, DataType = DataType.Optional};
var newConfigCalculated = await workspace.Query<AocConfiguration>().Where(x => x == newConfigExpected).ToArrayAsync();
newConfigCalculated.Count().Should().Be(1);

## RCU < NewAoc < CF

In [0]:
var aocConfigLog = await CheckAocConfigurationDefault( new[]{ new AocType{SystemName = "A1", DisplayName = "a1", Order = 45} } );
aocConfigLog.Errors.Any().Should().Be(false);

In [0]:
var newConfigExpected = (await workspace.Query<AocConfiguration>().ToArrayAsync()).First(x => 
                         x.AocType == AocTypes.RCU && x.Novelty == Novelties.I) with {AocType = "A1", Order = 31, DataType = DataType.Optional};
var newConfigCalculated = await workspace.Query<AocConfiguration>().Where(x => x == newConfigExpected).ToArrayAsync();
newConfigCalculated.Count().Should().Be(1);

## IA < NewAoc < YCU

In [0]:
var aocConfigLog = await CheckAocConfigurationDefault( new[]{ new AocType{SystemName = "A1", DisplayName = "a1", Order = 82} } );
aocConfigLog.Errors.Any().Should().Be(false);

In [0]:
var newConfigExpected = (await workspace.Query<AocConfiguration>().ToArrayAsync()).First(x => 
                         x.AocType == AocTypes.AU && x.Novelty == Novelties.I) with {AocType = "A1", Order = 61, DataType = DataType.Optional};
var newConfigCalculated = await workspace.Query<AocConfiguration>().Where(x => x == newConfigExpected).ToArrayAsync();
newConfigCalculated.Count().Should().Be(1);

In [0]:
var newConfigExpected = (await workspace.Query<AocConfiguration>().ToArrayAsync()).First(x => 
                         x.AocType == AocTypes.AU && x.Novelty == Novelties.N) with {AocType = "A1", Order = 131, DataType = DataType.Optional};
var newConfigCalculated = await workspace.Query<AocConfiguration>().Where(x => x == newConfigExpected).ToArrayAsync();
newConfigCalculated.Count().Should().Be(1);

## CRU < NewAoc < EV

In [0]:
var aocConfigLog = await CheckAocConfigurationDefault( new[]{ new AocType{SystemName = "A1", DisplayName = "a1", Order = 106} } );
aocConfigLog.Errors.Any().Should().Be(false);

In [0]:
var newConfigExpected = (await workspace.Query<AocConfiguration>().ToArrayAsync()).First(x => 
                         x.AocType == AocTypes.EV && x.Novelty == Novelties.I) with {AocType = "A1", Order = 81};
var newConfigCalculated = await workspace.Query<AocConfiguration>().Where(x => x == newConfigExpected).ToArrayAsync();
newConfigCalculated.Count().Should().Be(1);

In [0]:
var newConfigExpected = (await workspace.Query<AocConfiguration>().ToArrayAsync()).First(x => 
                         x.AocType == AocTypes.EV && x.Novelty == Novelties.N) with {AocType = "A1", Order = 131};
var newConfigCalculated = await workspace.Query<AocConfiguration>().Where(x => x == newConfigExpected).ToArrayAsync();
newConfigCalculated.Count().Should().Be(1);

## EV < NewAoc < WO

In [0]:
var aocConfigLog = await CheckAocConfigurationDefault( new[]{ new AocType{SystemName = "A1", DisplayName = "a1", Order = 116} } );
aocConfigLog.Errors.Any().Should().Be(false);

In [0]:
var newConfigExpected = (await workspace.Query<AocConfiguration>().ToArrayAsync()).First(x => 
                         x.AocType == AocTypes.EV && x.Novelty == Novelties.I) with {AocType = "A1", Order = 91};
var newConfigCalculated = await workspace.Query<AocConfiguration>().Where(x => x == newConfigExpected).ToArrayAsync();
newConfigCalculated.Count().Should().Be(1);

In [0]:
var newConfigExpected = (await workspace.Query<AocConfiguration>().ToArrayAsync()).First(x => 
                         x.AocType == AocTypes.EV && x.Novelty == Novelties.N) with {AocType = "A1", Order = 141};
var newConfigCalculated = await workspace.Query<AocConfiguration>().Where(x => x == newConfigExpected).ToArrayAsync();
newConfigCalculated.Count().Should().Be(1);

## WO < NewAoc < CL

In [0]:
var aocConfigLog = await CheckAocConfigurationDefault( new[]{ new AocType{SystemName = "A1", DisplayName = "a1", Order = 125} } );
aocConfigLog.Errors.Any().Should().Be(false);

In [0]:
var newConfigExpected = (await workspace.Query<AocConfiguration>().ToArrayAsync()).First(x => 
                         x.AocType == AocTypes.WO && x.Novelty == Novelties.C) with {AocType = "A1", Order = 171};
var newConfigCalculated = await workspace.Query<AocConfiguration>().Where(x => x == newConfigExpected).ToArrayAsync();
newConfigCalculated.Count().Should().Be(1);

## Two or more Aoc Config with same Order

In [0]:
var aocConfigLog = await CheckAocConfigurationDefault( new[]{ new AocType{SystemName = "A1", DisplayName = "a1", Order = 111},
                                                              new AocType{SystemName = "A2", DisplayName = "a1", Order = 112},
                                                              new AocType{SystemName = "A3", DisplayName = "a1", Order = 113},
                                                              new AocType{SystemName = "A4", DisplayName = "a1", Order = 114},
                                                              new AocType{SystemName = "A5", DisplayName = "a1", Order = 115} } );
(aocConfigLog.Errors.First().ToString() == "ActivityMessageNotification { Message = Two or more Aoc Configurations have the same Order. }").Should().Be(true);